# Carga de los datos

Este jupyter notebook esta generando los datos de un año en especifico (2012 el parquet part.0.parquet de rips) para coincidir con las tablas compartidas del  power bi (Tablas agregadas) 

In [3]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("GeneraciónTablasRips2").getOrCreate()

In [4]:
#spark.stop()

In [5]:
# Leer los archivos Parquet de un año 
df1_parquet = spark.read.parquet("/rawdata/rips/rips_parquet_2022/part.0.parquet")

df=df1_parquet

In [6]:
# Contar el número de registros en el DataFrame unido
total_registros = df.count()

# Listar las columnas del DataFrame unido
columnas = df.columns

# Mostrar resultados
print(f"Total de registros en el DataFrame unido: {total_registros}")
print(f"Columnas del DataFrame unido: {columnas}")

Total de registros en el DataFrame unido: 391525
Columnas del DataFrame unido: ['personaid', 'tipoeventoripsdesc', 'codigo', 'dxprincipal', 'dxegreso', 'finalidadprocedimientoscd', 'finalidadconsultacd', 'tipousuariocd', 'causaexternacd', 'prestador', 'estadosalidacd', 'tipodiagnosticoprincipalcd', 'codigoprocedimiento', 'municipiocd', 'fechaid', 'edad', 'sexodesc', 'costoconsulta', 'costoprocedimiento', 'netoapagarconsulta', 'numerodiasestancia', '__null_dask_index__']


In [7]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 391525


In [8]:
# Mostrar el esquema (columnas y tipos de datos)
df.printSchema()

# Listar las columnas
print(df.columns)

root
 |-- personaid: string (nullable = true)
 |-- tipoeventoripsdesc: string (nullable = true)
 |-- codigo: string (nullable = true)
 |-- dxprincipal: string (nullable = true)
 |-- dxegreso: string (nullable = true)
 |-- finalidadprocedimientoscd: string (nullable = true)
 |-- finalidadconsultacd: string (nullable = true)
 |-- tipousuariocd: string (nullable = true)
 |-- causaexternacd: string (nullable = true)
 |-- prestador: string (nullable = true)
 |-- estadosalidacd: string (nullable = true)
 |-- tipodiagnosticoprincipalcd: string (nullable = true)
 |-- codigoprocedimiento: string (nullable = true)
 |-- municipiocd: string (nullable = true)
 |-- fechaid: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- sexodesc: string (nullable = true)
 |-- costoconsulta: string (nullable = true)
 |-- costoprocedimiento: string (nullable = true)
 |-- netoapagarconsulta: string (nullable = true)
 |-- numerodiasestancia: string (nullable = true)
 |-- __null_dask_index__: long (nul

# Empieza el proceso de generación de los datos agrupados por atenciones

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df = df1

# Seleccionar solo las columnas deseadas (sin 'personaid_count' ya que no existe aún)
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd'
]
df_filtered = df.select(*columns_to_keep)

# Agregar una nueva columna con el recuento distintivo de 'personaid'
window_spec = Window.partitionBy('personaid')
df_filtered = df_filtered.withColumn('personaid_count', F.count('personaid').over(window_spec))

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered.show()

# Mostrar los nombres de las columnas filtradas para verificar
print("Nombres de las columnas filtradas:")
print(df_filtered.columns)

# Agrupar por la columna 'personaid'
grouped_df = df_filtered.groupBy('personaid').agg(
    F.first('codigoprocedimiento').alias('codigoprocedimiento'),
    F.first('tipoeventoripsdesc').alias('tipoeventoripsdesc'),
    F.first('fechaid').alias('fechaid'),
    F.first('sexodesc').alias('sexodesc'),
    F.first('tipodiagnosticoprincipalcd').alias('tipodiagnosticoprincipalcd'),
    F.first('edad').alias('edad'),
    F.first('dxegreso').alias('dxegreso'),
    F.first('dxprincipal').alias('dxprincipal'),
    F.first('municipiocd').alias('municipiocd'),
    F.first('personaid_count').alias('personaid_count')
)

# Mostrar las primeras filas del DataFrame agrupado para verificar
grouped_df.show()


24/08/16 19:52:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---------+-------------------+--------------------+--------+--------+--------------------------+----+--------+-----------+-----------+---------------+
|personaid|codigoprocedimiento|  tipoeventoripsdesc| fechaid|sexodesc|tipodiagnosticoprincipalcd|edad|dxegreso|dxprincipal|municipiocd|personaid_count|
+---------+-------------------+--------------------+--------+--------+--------------------------+----+--------+-----------+-----------+---------------+
|100012321|             903816|PROCEDIMIENTOS DE...|20221220|FEMENINO|                      null|  31|       0|       Z000|      41132|              3|
|100012321|             907106|PROCEDIMIENTOS DE...|20221220|FEMENINO|                      null|  31|       0|       Z000|      41132|              3|
|100012321|             903818|PROCEDIMIENTOS DE...|20221220|FEMENINO|                      null|  31|       0|       Z000|      41132|              3|
| 10002763|             890228|           CONSULTAS|20220810|FEMENINO|                  

+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|personaid|codigoprocedimiento|  tipoeventoripsdesc| fechaid| sexodesc|tipodiagnosticoprincipalcd|edad|dxegreso|dxprincipal|municipiocd|personaid_count|
+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|100012321|             903816|PROCEDIMIENTOS DE...|20221220| FEMENINO|                      null|  31|       0|       Z000|      41132|              3|
| 10002763|             890228|           CONSULTAS|20220810| FEMENINO|                         1|  61|       0|       I10X|      13001|              1|
|100043669|             904921|PROCEDIMIENTOS DE...|20220831| FEMENINO|                      null|  32|       0|       Z000|      47001|              5|
|100044684|             903026|PROCEDIMIENTOS DE...|20220806| FEMENINO|           

In [10]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = grouped_df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 188473


In [11]:
 df_final =grouped_df


In [12]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = df_final.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 188473


In [13]:
# Mostrar las primeras filas del DataFrame agrupado para verificar
df_final.show()

+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|personaid|codigoprocedimiento|  tipoeventoripsdesc| fechaid| sexodesc|tipodiagnosticoprincipalcd|edad|dxegreso|dxprincipal|municipiocd|personaid_count|
+---------+-------------------+--------------------+--------+---------+--------------------------+----+--------+-----------+-----------+---------------+
|100012321|             903816|PROCEDIMIENTOS DE...|20221220| FEMENINO|                      null|  31|       0|       Z000|      41132|              3|
| 10002763|             890228|           CONSULTAS|20220810| FEMENINO|                         1|  61|       0|       I10X|      13001|              1|
|100043669|             904921|PROCEDIMIENTOS DE...|20220831| FEMENINO|                      null|  32|       0|       Z000|      47001|              5|
|100044684|             903026|PROCEDIMIENTOS DE...|20220806| FEMENINO|           

Se comprueba los datos generados con las tablas, da correctamente el numero de atenciones filtrando por id vs el archivo de powerbi (personaid 10002763 , 100012321 , 10012258 , 100043669)

# Empieza el proceso de generación de tablas

## 1. Agrupado por año y mes

In [21]:
# Cargar el archivo Parquet en un DataFrame
df2 = df_final

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
df2 = df_final




In [23]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = df2.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 188473


In [24]:
# Mostrar el esquema (columnas y tipos de datos)
df2.printSchema()

# Listar las columnas
print(df2.columns)

root
 |-- personaid: string (nullable = true)
 |-- codigoprocedimiento: string (nullable = true)
 |-- tipoeventoripsdesc: string (nullable = true)
 |-- fechaid: string (nullable = true)
 |-- sexodesc: string (nullable = true)
 |-- tipodiagnosticoprincipalcd: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- dxegreso: string (nullable = true)
 |-- dxprincipal: string (nullable = true)
 |-- municipiocd: string (nullable = true)
 |-- personaid_count: long (nullable = true)

['personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid', 'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count']


# resultado final agrupado año y mes, según el numero de personas y atenciones

In [25]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año y mes
result_df = df_filtered.groupBy('Año', 'Mes').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Filtrar para mostrar los resultados de enero para comprobar
result_january = result_df.filter((F.col('Año') == 2022) & (F.col('Mes') == 1))

# Mostrar los resultados para enero
result_january.show()

# Mostrar las primeras filas del DataFrame completo para verificar
result_df.show()

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+------------------+--------------------+
| Año|Mes|Número_de_Personas|Número_de_Atenciones|
+----+---+------------------+--------------------+
|2022|  1|             11692|               28279|
+----+---+------------------+--------------------+



+----+---+------------------+--------------------+
| Año|Mes|Número_de_Personas|Número_de_Atenciones|
+----+---+------------------+--------------------+
|2022| 10|             18139|               34774|
|2022|  5|              6157|               10619|
|2022|  2|             10065|               21456|
|2022| 11|             26100|               51706|
|2022|  8|             23279|               58484|
|2022|  1|             11692|               28279|
|2022|  6|              5124|                8927|
|2022|  9|             15274|               29441|
|2022|  4|             15661|               38970|
|2022| 12|             37596|               69205|
|2022|  7|              6915|               12320|
|2022|  3|             12471|               27344|
+----+---+------------------+--------------------+

Nombres de las columnas agrupadas:
['Año', 'Mes', 'Número_de_Personas', 'Número_de_Atenciones']


Hay un ligero desfase al comparar los datos generados aqui al del powerbi , de 100 registros en promedio 

# 2. Agrupado por año, mes y sexo

In [26]:

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y sexodesc
result_df = df_filtered.groupBy('Año', 'Mes', 'sexodesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+-----------+------------------+--------------------+
| Año|Mes|   sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+------------------+--------------------+
|2022|  7|   FEMENINO|              4336|                7922|
|2022| 10|       null|                29|                  45|
|2022|  6|  MASCULINO|              1919|                3282|
|2022|  3|  MASCULINO|              4636|                9949|
|2022|  5|       null|                 8|                  11|
|2022|  2|       null|                13|                  22|
|2022| 11|       null|                39|                 107|
|2022|  2|   FEMENINO|              6168|               13336|
|2022|  8|       null|                46|                  90|
|2022| 11|   FEMENINO|             16569|               32885|
|2022|  9|  MASCULINO|              5524|               10526|
|2022| 12|   FEMENINO|             23296|               41979|
|2022| 12|NO DEFINIDO|               100|              

pasa lo mismo, difiere un poco de los datos filtrados del powerbi 

# 3. Agrupado por año, mes y municipio

In [27]:

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y municipioid
result_df = df_filtered.groupBy('Año', 'Mes', 'municipiocd').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+-----------+------------------+--------------------+
| Año|Mes|municipiocd|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+------------------+--------------------+
|2022|  7|      76377|                63|                 100|
|2022| 12|      05607|                10|                  16|
|2022| 12|      15238|                20|                  24|
|2022|  6|      15293|                 1|                   2|
|2022|  4|      73349|                 4|                   4|
|2022|  6|      08137|                 5|                   6|
|2022| 10|       null|               122|                 153|
|2022|  3|      25899|              1156|                3124|
|2022|  3|      15764|                24|                  44|
|2022|  8|      25740|                 3|                   4|
|2022| 10|      05674|                17|                  27|
|2022|  1|      05360|                 5|                  12|
|2022|  2|      15104|                 7|              

# 4. Agrupado por año, mes y tipoeventoripsdesc

In [28]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y tipoeventoripsdesc
result_df = df_filtered.groupBy('Año', 'Mes', 'tipoeventoripsdesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+--------------------+------------------+--------------------+
| Año|Mes|  tipoeventoripsdesc|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------------+------------------+--------------------+
|2022| 11|           CONSULTAS|              6181|                8715|
|2022| 12|           CONSULTAS|             10023|               13198|
|2022|  2|PROCEDIMIENTOS DE...|              8487|               19109|
|2022|  7|PROCEDIMIENTOS DE...|              4254|                8216|
|2022|  2|           CONSULTAS|              1578|                2347|
|2022| 10|PROCEDIMIENTOS DE...|             12057|               26129|
|2022|  5|           CONSULTAS|              1964|                3114|
|2022|  9|           CONSULTAS|              4719|                6224|
|2022|  3|           CONSULTAS|              1630|                2569|
|2022|  3|PROCEDIMIENTOS DE...|             10841|               24775|
|2022|  7|           CONSULTAS|              2661|              

# 5. Agrupado por año, mes y tipodiagnosticoprincipalcd

In [29]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y tipodiagnosticoprincipalcd
result_df = df_filtered.groupBy('Año', 'Mes', 'tipodiagnosticoprincipalcd').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+--------------------------+------------------+--------------------+
| Año|Mes|tipodiagnosticoprincipalcd|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------------------+------------------+--------------------+
|2022| 10|                      null|             12057|               26129|
|2022|  9|                         1|              3692|                4837|
|2022|  6|                         1|              1494|                2103|
|2022|  7|                         3|               293|                 485|
|2022|  2|                         2|               259|                 414|
|2022|  6|                         2|               455|                 627|
|2022|  7|                         2|               416|                 658|
|2022|  5|                      null|              4193|                7505|
|2022| 10|                         3|              1572|                1953|
|2022|  5|                         3|               210|        

# 6. Agrupado por año, mes y dxprincipal

In [30]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y dxprincipal
result_df = df_filtered.groupBy('Año', 'Mes', 'dxprincipal').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+------------------+--------------------+
| Año|Mes|dxprincipal|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+------------------+--------------------+
|2022| 12|       K046|                27|                  37|
|2022| 11|       O269|                 7|                  13|
|2022|  8|       G138|                 1|                   1|
|2022| 10|       M401|                 5|                   6|
|2022|  8|       R100|                 6|                   7|
|2022| 10|       E079|                 4|                   7|
|2022|  7|       P229|                 2|                   3|
|2022|  2|       E669|                13|                  23|
|2022|  9|       I509|                 3|                   4|
|2022|  7|       S025|                 8|                  10|
|2022|  5|       S708|                 1|                   8|
|2022|  9|       N760|                16|                  21|
|2022| 10|       N771|                28|              

# 7. Agrupado por año, mes y codigoprocedimiento

In [31]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'codigoprocedimiento').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-------------------+------------------+--------------------+
| Año|Mes|codigoprocedimiento|Número_de_Personas|Número_de_Atenciones|
+----+---+-------------------+------------------+--------------------+
|2022| 12|             682101|                11|                  11|
|2022|  6|             881401|                12|                  14|
|2022| 12|             890750|                21|                  25|
|2022|  2|             901303|                 1|                   1|
|2022| 10|             903883|                23|                  53|
|2022|  3|             873420|                32|                  60|
|2022|  7|             865101|                 6|                   6|
|2022|  3|             911003|                 2|                   4|
|2022| 12|             873423|                 2|                   2|
|2022|  8|             882317|                21|                  71|
|2022|  2|             907008|                40|                  94|
|2022|

se verifican algunos resultados de enero, coinciden perfectamente

# 8. Agrupado por año, mes, municipio y sexo

In [33]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'municipiocd','sexodesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+---------+------------------+--------------------+
| Año|Mes|municipiocd| sexodesc|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+---------+------------------+--------------------+
|2022| 10|      47189| FEMENINO|               618|                1553|
|2022|  7|      70678| FEMENINO|                 6|                  10|
|2022|  8|      25307| FEMENINO|                39|                  52|
|2022|  1|      23417| FEMENINO|                 1|                   1|
|2022|  4|      15764|MASCULINO|                 9|                  22|
|2022|  9|      41668| FEMENINO|                10|                  14|
|2022|  1|      15407| FEMENINO|                 6|                   7|
|2022|  9|      94001|MASCULINO|                 2|                   2|
|2022|  7|       null| FEMENINO|                36|                  44|
|2022|  2|      25269| FEMENINO|               399|                1428|
|2022|  9|      70713|MASCULINO|                57|

da correctamente al verificar unos registros de enero

# 9. Agrupado por año, mes, tipoeventorips y sexo

In [ ]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'tipoeventoripsdesc','sexodesc').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

difiere muy poco de los datos filtrados del powerbi 

# 10. Agrupado por año, mes, edad

In [ ]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'edad').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

se comprueba los de enero y coincide exactamente con los del powerbi

# 11. Agrupado por año, mes, municipiocd, tipoeventoripsdesc, dxprincipal, sexodesc, edad

In [34]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes', 'municipiocd','tipoeventoripsdesc','dxprincipal','sexodesc','edad').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+--------------------+-----------+-----------+----+------------------+--------------------+
| Año|Mes|municipiocd|  tipoeventoripsdesc|dxprincipal|   sexodesc|edad|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+--------------------+-----------+-----------+----+------------------+--------------------+
|2022|  3|      19001|PROCEDIMIENTOS DE...|       Z000|  MASCULINO|  18|                 1|                   1|
|2022| 12|      76869|PROCEDIMIENTOS DE...|       R688|   FEMENINO|  16|                 3|                   4|
|2022|  2|      15759|PROCEDIMIENTOS DE...|       Z136|   FEMENINO|  60|                 9|                  18|
|2022| 11|      70708|PROCEDIMIENTOS DE...|       N390|   FEMENINO|  16|                 2|                   3|
|2022| 12|      47798|PROCEDIMIENTOS DE...|       Z008|  MASCULINO|  14|                 1|                   1|
|2022| 12|      76001|           CONSULTAS|       Z003|  MASCULINO|  13|                 4|     

Se valida estos datos (2022|  1|15001|PROCEDIMIENTOS DE...|H813|FEMENINO|  63|1|3| 
|2022|  1|15759|PROCEDIMIENTOS DE...|K036|MASCULINO|18|1|3|) corresponde exactamente


# 12. Agrupado por año, mes, sexodesc, edad

In [35]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'personaid', 'codigoprocedimiento', 'tipoeventoripsdesc', 'fechaid',
    'sexodesc', 'tipodiagnosticoprincipalcd', 'edad', 'dxegreso', 'dxprincipal', 'municipiocd', 'personaid_count'
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'fechaid' a tipo fecha si es necesario
df_filtered = df_filtered.withColumn('fechaid', F.to_date('fechaid', 'yyyyMMdd'))

# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('fechaid'))
df_filtered = df_filtered.withColumn('Mes', F.month('fechaid'))

# Agrupar por año, mes y codigoprocedimiento
result_df = df_filtered.groupBy('Año', 'Mes','sexodesc', 'edad').agg(
    F.countDistinct('personaid').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------+----+------------------+--------------------+
| Año|Mes|   sexodesc|edad|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------+----+------------------+--------------------+
|2022|  3|   FEMENINO|  47|                93|                 172|
|2022|  6|   FEMENINO|  17|                61|                  87|
|2022| 11|   FEMENINO|  73|               155|                 392|
|2022|  3|   FEMENINO|  62|               127|                 287|
|2022|  4|  MASCULINO|  22|                69|                 191|
|2022|  1|  MASCULINO|  19|                13|                  27|
|2022|  4|  MASCULINO|  79|                47|                 167|
|2022|  9|  MASCULINO|  33|                39|                  66|
|2022|  3|   FEMENINO|  20|                83|                 153|
|2022|  5|NO DEFINIDO|   9|                17|                  19|
|2022|  8|  MASCULINO| 100|                 3|                   6|
|2022|  8|  MASCULINO|  21|                70|  

se validan estos datos: |2022|1|MASCULINO|19|13|27| 
                        |2022|1|MASCULINO|93|10|34| Corresponden exactamente al tablero power bi

In [36]:
spark.stop()